In [ ]:
# セル 1: 必要なライブラリのインポート
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
import torch
import numpy as np
from pathlib import Path
import threading
import queue
import time
import os
from loguru import logger
import io

# fish-speech関連のモジュールをインポート
# (fish-speechがインストールされ、パスが通っている必要があります)
# 必要に応じて sys.path に fish-speech のパスを追加してください
# import sys
# sys.path.append('/path/to/fish-speech') # 例
try:
    from fish_speech.inference_engine import TTSInferenceEngine
    from fish_speech.models.text2semantic.inference import launch_thread_safe_queue
    from fish_speech.models.vqgan.inference import load_model as load_decoder_model
    from fish_speech.utils.schema import ServeTTSRequest
except ImportError as e:
    print(f"Error importing fish-speech modules: {e}")
    print("Please ensure fish-speech is installed and accessible in your Python path.")
    # ここで処理を中断するか、ダミーの関数を定義するなどの対応が必要です
    raise e

print("必要なライブラリをインポートしました。")

In [ ]:
# セル 2: 設定値 (モデルパスなどを環境に合わせて変更してください)

# --- ユーザー設定 ---
LLAMA_CHECKPOINT_PATH = Path("./checkpoints/fish-speech-1.5") # LLAMAモデルのパス
DECODER_CHECKPOINT_PATH = Path("./checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth") # VQGANモデルのパス
DECODER_CONFIG_NAME = "firefly_gan_vq" # VQGANモデルの設定名
# --------------------

# デバイス設定 (自動検出)
if torch.cuda.is_available():
    DEVICE = "cuda"
    PRECISION = torch.bfloat16 # または torch.float16
elif torch.backends.mps.is_available():
    DEVICE = "mps"
    PRECISION = torch.float16 # MPSはbfloat16をサポートしない場合がある
else:
    DEVICE = "cpu"
    PRECISION = torch.float32

COMPILE_MODEL = True # モデルコンパイルを有効にするか (高速化のため推奨)

# WindowsかつCompile=Trueの場合、警告を出す (動作しない可能性があるため)
if os.name == 'nt' and COMPILE_MODEL:
    print("警告: Windows環境でのモデルコンパイルは不安定な場合があります。エラーが発生する場合は COMPILE_MODEL = False を試してください。")

# Make einx happy (fish-speechが必要とする可能性あり)
os.environ["EINX_FILTER_TRACEBACK"] = "false"

print(f"デバイス: {DEVICE}")
print(f"精度: {PRECISION}")
print(f"モデルコンパイル: {COMPILE_MODEL}")
print(f"LLAMAモデルパス: {LLAMA_CHECKPOINT_PATH}")
print(f"VQGANモデルパス: {DECODER_CHECKPOINT_PATH}")

In [ ]:
# セル 3: モデルの読み込みとコンパイル (ログ出力修正)

# ★ ログ表示用のOutputウィジェットを先に定義 (セル4の前に移動しても良い)
log_output = widgets.Output(layout=widgets.Layout(height='100px', border='1px solid gray', overflow_y='scroll'))

# ★ ログメッセージを出力する関数
def log_message(msg, level="info"):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    with log_output:
        if level == "error":
            print(f"<font color='red'>[{timestamp}] ERROR: {msg}</font>")
        else:
            print(f"[{timestamp}] INFO: {msg}")

# --- モデル読み込み処理 ---
log_message("モデルの読み込みとコンパイルを開始します。時間がかかる場合があります...")
start_time = time.time()

# 1. LLAMAモデルを別スレッドで読み込み&コンパイル
try:
    log_message("LLAMAモデルを読み込んでいます...")
    llama_queue = launch_thread_safe_queue(
        checkpoint_path=LLAMA_CHECKPOINT_PATH,
        device=DEVICE,
        precision=PRECISION,
        compile=COMPILE_MODEL,
    )
    log_message("LLAMAモデルの読み込み完了。")
except Exception as e:
    log_message(f"LLAMAモデルの読み込み中にエラーが発生しました: {e}", level="error")
    if COMPILE_MODEL:
        log_message("コンパイルなしで再試行します (COMPILE_MODEL = False)...")
        COMPILE_MODEL = False
        llama_queue = launch_thread_safe_queue(
            checkpoint_path=LLAMA_CHECKPOINT_PATH,
            device=DEVICE,
            precision=PRECISION,
            compile=COMPILE_MODEL,
        )
        log_message("LLAMAモデルの読み込み完了 (コンパイルなし)。")
    else:
        raise e

# 2. VQGANデコーダーモデルの読み込み
log_message("VQGANデコーダーモデルを読み込んでいます...")
decoder_model = load_decoder_model(
    config_name=DECODER_CONFIG_NAME,
    checkpoint_path=DECODER_CHECKPOINT_PATH,
    device=DEVICE,
)
log_message("VQGANデコーダーモデルの読み込み完了。")

# 3. TTS推論エンジンの初期化
log_message("TTS推論エンジンを初期化しています...")
inference_engine = TTSInferenceEngine(
    llama_queue=llama_queue,
    decoder_model=decoder_model,
    compile=COMPILE_MODEL,
    precision=PRECISION,
)
log_message("TTS推論エンジンの初期化完了。")

# 4. ウォームアップ実行
log_message("モデルのウォームアップを実行中...")
try:
    # ウォームアップのリクエストを少し調整
    warmup_req = ServeTTSRequest(text="音声合成エンジンのウォームアップ中です。")
    _ = list(inference_engine.inference(warmup_req))
    log_message("ウォームアップ完了。")
except Exception as e:
    log_message(f"ウォームアップ中にエラーが発生しました: {e}", level="error")
    log_message("アプリは起動しますが、初回の音声生成に時間がかかる可能性があります。")

end_time = time.time()
log_message(f"モデルの準備完了。所要時間: {end_time - start_time:.2f} 秒")

# グローバル変数としてエンジンを保持
tts_engine_global = inference_engine

# ★ セル3の最後にログ出力エリアを表示
display(log_output)

In [ ]:
# セル 4: ipywidgets UIの定義と表示 (ログエリア追加)

# --- UI要素の定義 ---
app_title = widgets.HTML("<h1>GMC Speech</h1>")
text_input = widgets.Textarea(
    placeholder='ここにテキストを入力するか、下のボタンからファイルをアップロードして内容を編集してください。',
    layout=widgets.Layout(width='95%', height='150px')
)
file_upload = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='テキスト読込(&編集)',
    tooltip='テキストファイルを読み込み、テキストエリアに表示して編集できます'
)
generate_button = widgets.Button(
    description='音声生成',
    button_style='success',
    tooltip='入力/編集されたテキストから音声を生成します',
    icon='play',
    layout=widgets.Layout(width='auto')
)
status_label = widgets.Label(value="準備完了") # 初期ステータス
audio_output = widgets.Output()

# --- UIレイアウト ---
ui_layout = widgets.VBox([
    app_title,
    text_input,
    widgets.HBox([file_upload, generate_button]),
    status_label,
    audio_output,
    widgets.HTML("<hr><b>ログ:</b>"), # ログエリアの区切り線とタイトル
    log_output # ★ ログ表示エリアをレイアウトに追加
])

# --- UI表示 ---
display(ui_layout)

# log_message("UIを表示しました。") # log_outputはセル3の最後で表示されるため、ここでは不要

In [ ]:
# セル 5: イベントハンドラの定義 (ファイル読込、完了表示タイミング、ログ修正)

# ファイルアップロード時の処理 (修正版)
def on_file_upload_change(change):
    # `change['new']` は現在のウィジェットの値 (ファイルのタプル)
    # `change['old']` は変更前の値
    uploaded_files_tuple = change['new']
    if uploaded_files_tuple: # タプルが空でなければファイルが選択された
        uploaded_file_info = uploaded_files_tuple[0] # 最初のファイルを取得
        file_name = uploaded_file_info['name']
        file_content = uploaded_file_info['content'] # これは memoryview または bytes

        log_message(f"ファイル '{file_name}' が選択されました。読み込み中...")
        try:
            # content が memoryview の場合は bytes に変換
            if isinstance(file_content, memoryview):
                content_bytes = file_content.tobytes()
            else:
                content_bytes = file_content

            # UTF-8でデコードを試みる
            content = content_bytes.decode('utf-8')
            text_input.value = content # テキストエリアに設定
            status_label.value = f"ファイル '{file_name}' を読み込みました。内容は編集可能です。"
            log_message(f"ファイル '{file_name}' の内容をテキストエリアに表示しました。")
        except UnicodeDecodeError:
            # 他のエンコーディング (Shift_JISなど) で試す場合
            try:
                log_message("UTF-8でのデコードに失敗、Shift_JISで再試行...")
                content = content_bytes.decode('shift_jis')
                text_input.value = content
                status_label.value = f"ファイル '{file_name}' を読み込みました(Shift_JIS)。内容は編集可能です。"
                log_message(f"ファイル '{file_name}' の内容をテキストエリアに表示しました(Shift_JIS)。")
            except Exception as e:
                status_label.value = f"ファイルの読み込み/デコードに失敗しました: {e}"
                log_message(f"ファイルの読み込み/デコードに失敗しました: {file_name}, Error: {e}", level="error")
        except Exception as e:
            status_label.value = f"ファイルの処理中にエラーが発生しました: {e}"
            log_message(f"ファイルの処理中にエラーが発生しました: {file_name}, Error: {e}", level="error")
        finally:
            # ★ アップロードウィジェットの値をクリア (コールバック内で値を変更して再度トリガーさせないように注意)
            #   None を設定するとリセットされることが多い
            if file_upload.value is not None:
                # イベントループの問題を避けるため、直接Noneを代入
                file_upload.value = None
                # widgetの値を直接変更した場合、observeが再度呼ばれる可能性があるため注意
                # 次回のアップロードのためにクリアするだけならこれでOKな場合が多い

    # ファイル選択がキャンセルされた場合やクリアされた場合 (タプルが空になる)
    # else:
    #    log_message("ファイル選択がキャンセルされました。")
    #    pass

# 音声生成ボタンクリック時の処理 (完了表示タイミング、ログ修正)
def on_generate_button_clicked(b):
    # 既存の音声出力をクリア
    with audio_output:
        clear_output(wait=True)

    input_text = text_input.value.strip()
    if not input_text:
        status_label.value = "テキストが入力されていません。"
        log_message("音声生成をスキップ: テキスト未入力", level="error")
        return

    # ボタンを無効化し、ステータスを更新
    generate_button.disabled = True
    status_label.value = "音声生成中... (テキストの長さによっては時間がかかります)"
    log_message(f"音声生成開始: テキスト長={len(input_text)}")

    try:
        start_gen_time = time.time()

        # TTS推論リクエストを作成
        req = ServeTTSRequest(text=input_text)

        # グローバル変数から推論エンジンを取得
        engine = tts_engine_global

        # 音声生成を実行
        generated_audio_data = None
        sample_rate = 44100

        log_message("推論エンジン呼び出し中...")
        for result in engine.inference(req):
            if result.code == "final" and result.audio is not None:
                log_message("推論エンジンから最終音声データ受信。")
                sample_rate, generated_audio_data = result.audio
                break
            elif result.code == "error":
                log_message(f"推論中にエラー発生: {result.error}", level="error")
                raise result.error
            elif result.code == "header":
                log_message("音声ストリームヘッダー受信。")
            elif result.code == "segment":
                log_message("音声セグメント受信。") # ストリーミングしない場合は通常通らない

        end_gen_time = time.time()
        gen_duration = end_gen_time - start_gen_time
        log_message(f"推論エンジン処理完了。({gen_duration:.2f} 秒)")

        if generated_audio_data is not None:
            # ★ 音声データをIPython.display.Audioで表示
            log_message("音声プレーヤーを表示します...")
            with audio_output:
                display(Audio(data=generated_audio_data, rate=sample_rate))
            # ★ 音声表示後にステータスを更新
            status_label.value = f"音声生成完了！ ({gen_duration:.2f} 秒)"
            log_message(f"音声生成完了。再生時間: {len(generated_audio_data)/sample_rate:.2f}秒")
        else:
            status_label.value = "音声の生成に失敗しました（データがありません）。"
            log_message("音声生成失敗: 推論エンジンからデータが返されませんでした。", level="error")

    except Exception as e:
        status_label.value = f"エラーが発生しました: {e}"
        log_message(f"音声生成中に予期せぬエラー: {e}", level="error")
        import traceback
        log_message(traceback.format_exc(), level="error") # 詳細なトレースバックをログへ

    finally:
        # ボタンを再度有効化
        generate_button.disabled = False
        log_message("音声生成ボタンを有効化しました。")

# log_message("イベントハンドラを定義しました。") # log_outputはセル3の最後で表示されるため、ここでは不要

In [ ]:
# セル 6: イベントハンドラとウィジェットの連携設定

# FileUploadウィジェットの値が変更されたらon_file_upload_changeを呼び出す
file_upload.observe(on_file_upload_change, names='value')

# Buttonウィジェットがクリックされたらon_generate_button_clickedを呼び出す
generate_button.on_click(on_generate_button_clicked)

print("イベントハンドラを設定しました。アプリを使用できます。")